# **Advanced Chatbot in Python (Complete Code)**

This chatbot uses **NLP (Natural Language Processing)** with **intent recognition**, **contextual responses**, and **machine learning** for better conversation handling. It includes:

- **Intent detection** (using keyword matching & ML)
- **Contextual memory** (remembers past interactions)
- **Dynamic responses** (API integration for weather/news)
- **GUI (Tkinter) & CLI versions**

---

## **1. Required Libraries**

```bash
pip install nltk numpy tensorflow tkinter requests
```

---

## **2. NLP-Based Chatbot (Using NLTK & TensorFlow)**

### **a. Training Data (`intents.json`)**

```json
{
  "intents": [
    {
      "tag": "greeting",
      "patterns": ["Hi", "Hello", "Hey", "Good morning"],
      "responses": ["Hello!", "Hi there!", "How can I help you?"]
    },
    {
      "tag": "goodbye",
      "patterns": ["Bye", "See you later", "Goodbye"],
      "responses": ["Goodbye!", "See you soon!", "Take care!"]
    },
    {
      "tag": "weather",
      "patterns": ["What's the weather?", "Is it raining?", "Weather forecast"],
      "responses": ["I can check the weather. Please tell me your city."],
      "context": ["weather_city"]
    },
    {
      "tag": "news",
      "patterns": ["Latest news", "What's happening?", "News update"],
      "responses": ["Fetching the latest news..."]
    }
  ]
}
```

### **b. Complete Python Code**

```python
import json
import random
import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import SGD
import pickle
import requests

# Download NLTK data
nltk.download('punkt')
nltk.download('wordnet')

# Initialize lemmatizer
lemmatizer = WordNetLemmatizer()

# Load intents
intents = json.loads(open('intents.json').read())

# Preprocess data
words = []
classes = []
documents = []
ignore_chars = ['?', '!', '.', ',']

for intent in intents['intents']:
    for pattern in intent['patterns']:
        word_list = nltk.word_tokenize(pattern)
        words.extend(word_list)
        documents.append((word_list, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

words = [lemmatizer.lemmatize(word.lower()) for word in words if word not in ignore_chars]
words = sorted(set(words))
classes = sorted(set(classes))

pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))

# Create training data
training = []
output_empty = [0] * len(classes)

for doc in documents:
    bag = []
    word_patterns = doc[0]
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]

    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)

    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])

random.shuffle(training)
training = np.array(training, dtype=object)

train_x = list(training[:, 0])
train_y = list(training[:, 1])

# Build neural network model
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

sgd = SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5')

# Chatbot response function
def get_response(user_input):
    # Tokenize and lemmatize input
    input_words = nltk.word_tokenize(user_input)
    input_words = [lemmatizer.lemmatize(word.lower()) for word in input_words]

    # Create bag of words
    bag = [0] * len(words)
    for word in input_words:
        if word in words:
            bag[words.index(word)] = 1

    # Predict intent
    prediction = model.predict(np.array([bag]))[0]
    threshold = 0.7
    results = [[i, p] for i, p in enumerate(prediction) if p > threshold]
    results.sort(key=lambda x: x[1], reverse=True)

    if results:
        intent_tag = classes[results[0][0]]
        for intent in intents['intents']:
            if intent['tag'] == intent_tag:
                return random.choice(intent['responses'])
    return "I didn't understand that. Can you rephrase?"

# GUI Version (Tkinter)
import tkinter as tk
from tkinter import scrolledtext

class ChatbotGUI:
    def __init__(self, master):
        self.master = master
        master.title("Advanced Chatbot")

        self.chat_history = scrolledtext.ScrolledText(master, width=50, height=20)
        self.chat_history.pack(pady=10)

        self.user_input = tk.Entry(master, width=40)
        self.user_input.pack(pady=5)

        self.send_button = tk.Button(master, text="Send", command=self.send_message)
        self.send_button.pack()

    def send_message(self):
        user_text = self.user_input.get()
        self.chat_history.insert(tk.END, f"You: {user_text}\n")
        self.user_input.delete(0, tk.END)

        bot_response = get_response(user_text)
        self.chat_history.insert(tk.END, f"Bot: {bot_response}\n")

# Run the chatbot
print("Chatbot is ready! Type 'quit' to exit.")
root = tk.Tk()
gui = ChatbotGUI(root)
root.mainloop()
```

---

## **3. Key Features**

### **a. Intent Recognition**

- Uses **NLTK** for tokenization & lemmatization.
- **Neural Network (Keras)** for classifying user intents.

### **b. Contextual Responses**

- Handles follow-up questions (e.g., weather → city input).

### **c. API Integration (Example: Weather)**

```python
def get_weather(city):
    API_KEY = "your_api_key"
    url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid={API_KEY}"
    response = requests.get(url).json()
    if response.get("weather"):
        return f"Weather in {city}: {response['weather'][0]['description']}"
    return "Could not fetch weather data."
```

### **d. GUI & CLI Support**

- **Tkinter** for a user-friendly interface.
- **Terminal-based** version also available.

---

## **4. How to Improve Further**

✅ **Add more training data** (expand `intents.json`).  
✅ **Integrate speech recognition** (using `speech_recognition`).  
✅ **Deploy as a web app** (Flask/Django).  
✅ **Use Transformers (GPT-3)** for smarter responses.

---

## **5. Run the Chatbot**

```bash
python chatbot.py
```

(Ensure `intents.json` is in the same directory.)

---

## **Final Thoughts**

This chatbot:

- **Understands user intent** using NLP.
- **Remembers context** (e.g., asks for a city after weather query).
- **Can be extended** (APIs, voice, web interface).


In [1]:
import nltk
from nltk.chat.util import Chat, reflections

# Define patterns and responses
pairs = [
    [
        r"hi|hello|hey",
        ["Hello!", "Hi there!", "Hey!"]
    ],
    [
        r"how are you ?",
        ["I'm good, how about you?", "Doing great! How about yourself?"]
    ],
    [
        r"(.*) your name ?",
        ["I'm a chatbot!", "Call me ChatGPT!"]
    ],
    [
        r"quit",
        ["Goodbye!", "See you later!", "Have a great day!"]
    ]
]

# Create chatbot
chatbot = Chat(pairs, reflections)

def chat():
    print("Hello! I'm your chatbot. Type 'quit' to exit.")
    while True:
        user_input = input("")
        if user_input.lower() == 'quit':
            print("Goodbye!")
            break
        response = chatbot.respond(user_input)
        print(response)

if __name__ == "__main__":
    chat()


ModuleNotFoundError: No module named 'nltk'

Creating an advanced chatbot in Python involves several aspects, including natural language processing (NLP), machine learning models, and frameworks for handling conversational logic. A complete Python code for an advanced chatbot would require integrating libraries for NLP, such as **NLTK**, **spaCy**, or using pre-trained models from **Hugging Face's Transformers**.

Below is a more advanced Python chatbot that uses a **Transformer-based model** from the Hugging Face library, along with **Flask** for creating a web interface. This chatbot will leverage a pre-trained model for better understanding and generating responses.

### **Prerequisites:**

1. Install necessary libraries:

   ```bash
   pip install transformers
   pip install torch
   pip install flask
   ```

2. The chatbot will use a pre-trained model from the Hugging Face library. We'll use a model like **DialoGPT** for conversational capabilities.

### **Complete Python Code for an Advanced Chatbot:**

```python
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from flask import Flask, request, jsonify

# Load pre-trained model and tokenizer from Hugging Face
model_name = "microsoft/DialoGPT-medium"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Initialize Flask app
app = Flask(__name__)

# Keep track of the conversation history
chat_history_ids = None

# Helper function to generate responses
def generate_response(input_text):
    global chat_history_ids

    # Tokenize user input and encode it into tensors
    new_user_input_ids = tokenizer.encode(input_text + tokenizer.eos_token, return_tensors='pt')

    # Append new user input to the history
    if chat_history_ids is None:
        chat_history_ids = new_user_input_ids
    else:
        chat_history_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1)

    # Generate a response from the model
    bot_output_ids = model.generate(chat_history_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id,
                                    top_k=50, top_p=0.95, temperature=0.7, no_repeat_ngram_size=3, do_sample=True)

    # Decode the bot's response and update the history
    chat_history_ids = bot_output_ids
    bot_response = tokenizer.decode(bot_output_ids[:, chat_history_ids.shape[-1]:][0], skip_special_tokens=True)

    return bot_response

# Route to handle chatbot requests via POST method
@app.route("/chat", methods=["POST"])
def chat():
    # Get the user input from the POST request
    user_input = request.json.get("message")

    if user_input:
        # Generate response from the model
        bot_response = generate_response(user_input)
        return jsonify({"response": bot_response})
    else:
        return jsonify({"response": "Sorry, I didn't understand that."})

# Main route for testing the chatbot
@app.route("/", methods=["GET"])
def home():
    return "Chatbot is running. Send a POST request to /chat."

if __name__ == "__main__":
    # Run the Flask app
    app.run(debug=True)
```

### **Explanation of the Code:**

1. **Model and Tokenizer Setup**:

   - We load the **DialoGPT-medium** model and its tokenizer from the Hugging Face `transformers` library. The tokenizer converts the text input into token IDs that the model can process, and the model generates the response based on the input.
   - The `AutoModelForCausalLM` is used for causal language modeling (chatbots and text generation).

2. **Conversation History**:

   - The `chat_history_ids` variable is used to keep track of the conversation history. This allows the model to generate responses that are contextually aware of previous messages.

3. **Flask API Setup**:
   - The chatbot runs as a simple Flask web service. You can send a POST request with a message, and the bot will respond.
   - The endpoint `/chat` listens for POST requests with JSON data that includes the message from the user.
4. **Response Generation**:
   - The function `generate_response` processes the user's input, appends it to the conversation history, and then uses the model to generate a response.
   - The response is then decoded and returned as a JSON object.

### **Running the Chatbot Locally:**

1. Save the code above into a Python file (e.g., `chatbot.py`).
2. Run the chatbot with:

   ```bash
   python chatbot.py
   ```

   This will start a Flask server on `http://127.0.0.1:5000/`.

3. Use a REST client like **Postman** or **cURL** to interact with the chatbot by sending POST requests.

   Example `cURL` command to chat with the bot:

   ```bash
   curl -X POST -H "Content-Type: application/json" -d '{"message": "Hello, chatbot!"}' http://127.0.0.1:5000/chat
   ```

### **Sample Output:**

If you send a message like:

```json
{
  "message": "Hi, how are you?"
}
```

The bot might respond with:

```json
{
  "response": "Hi! I'm doing great. How can I assist you today?"
}
```

### **Features and Customization:**

- **Memory Management**: The `chat_history_ids` allows the chatbot to remember previous conversations, which makes it more interactive and context-aware.
- **Advanced Features**:
  - **Multi-turn Conversations**: By tracking `chat_history_ids`, you can enable the bot to handle multi-turn conversations.
  - **Temperature, Top-K, and Top-P Sampling**: The model’s response generation can be adjusted by tweaking the parameters like temperature (for randomness), top-k (limiting the candidate words), and top-p (nucleus sampling).

### **Conclusion:**

This chatbot setup is based on the **DialoGPT-medium** model and provides a simple yet powerful conversational interface. By using Flask, we’ve created a web API that allows you to interact with the chatbot via HTTP requests. You can improve this further by adding features like sentiment analysis, more sophisticated state management, or integrating it with a front-end for a more interactive experience.
